# Eclat
Eclat steht für **E**quivalence **Cla**ss **T**ransformation und ist ein weiterer Algorithmus für das Auffinden häufiger Muster. Im Gegensatz zum Apriori-Algorithmus, der einen horizontalen Ansatz verfolgt, verwendet Eclat einen vertikalen Ansatz zur Mustererkennung.

In [ ]:
from tui_dsmt import OrderedSet
from tui_dsmt.fpm import characters

## Inhaltsverzeichnis
- [Vertikale Datenrepräsentation](#Vertikale-Datenrepräsentation)
- [Vorbereitung der Traversierung und Äquivalenzklassen](#Vorbereitung-der-Traversierung-und-Äquivalenzklassen)
- [Bottom-Up-Lattice-Traversierung](#Bottom-Up-Lattice-Traversierung)

## Vertikale Datenrepräsentation
Zur Verwendung des Eclat-Algorithmus muss zuerst die Liste der Transaktionen in eine andere Form umgewandelt werden. Die bisher untersuchten Einträge der Transaktionsdatenbanken bestanden grundsätzlich aus einer Transaktionsnummer und einer Menge an enthaltenen Items:

In [ ]:
characters

Ziel ist es nun für jedes Item eine Menge an Transaktionsnummern zu speichern, in denen dieses Item vorkommt:

In [ ]:
all_items = {}

for tid, itemset in characters:
    for item in itemset:
        if (item,) not in all_items:
            all_items[(item,)] = OrderedSet()

        all_items[(item,)].add(tid)

all_items

## Vorbereitung der Traversierung und Äquivalenzklassen
Auch für Eclat wird ein minimaler Support verwendet, um frühzeitig Kandidaten auszuschließen.

In [ ]:
min_supp = 3

Das Filtern nach dem minimalen Support ist sehr einfach, da für jedes Item alle zugehörigen Transaktionen bereits in einer Menge gespeichert sind.

In [ ]:
for item, tids in all_items.items():
    if len(tids) >= min_supp:
        print(item)

Um nun zwei Itemsets zu kombinieren, müssen diese ein gemeinsames Präfix besitzen. Das bedeutet, dass bei Itemsets der Größe $k$ die ersten $k-1$ Items identisch sein müssen. Als **Äquivalenzklasse** wird dabei immer eine Menge aller Itemsets bezeichnet, die das gleiche Präfix besitzen und somit den Suchraum partitionieren.

Zunächst ist das Präfix leer und alle $1$-Itemsets teilen sich dieses leere Präfix. Die Kombination zweier $1$-Itemsets ist daher sehr einfach. (Das kaufmännische Und `&` steht in Python für den Schnitt beider Mengen!)

In [ ]:
cf = all_items[('c',)] & all_items[('f',)]
cf

In [ ]:
cp = all_items[('c',)] & all_items[('p',)]
cp

Das Ergebnis ist also jeweils eine Menge von Transaktionen, die zuvor beiden zur Konstruktion verwendeten $1$-Itemsets zugeordnet wurden. Der Schnitt enthält somit alle Transaktionen, in denen das $2$-Itemset vollständig enthalten ist.

Um ein $3$-Itemset zu erzeugen, werden zwei $2$-Itemsets aus der selben Äquivalenzklasse - also mit dem selben $1$-Präfix - kombiniert. In der nachfolgenden Zelle ist das Präfix dementsprechend `c`. Der Schnitt enthält wieder alle Transaktionsnummern, die das entstehende $3$-Itemset vollständig enthalten.

In [ ]:
cfp = cf & cp
cfp

Die Kombinationen nur innerhalb einer Äquivalenzklasse zu bilden verhindert, dass Kandidaten mehrfach erzeugt werden. `cfp` könnte beispielsweise auch durch Kombination der Itemsets `cf` und `fp` gefunden werden. Diese Kombination wird aber nicht in Betracht gezogen, da die beiden $2$-Itemsets sich nicht in der selben Äquivalenzklasse befinden beziehungsweise sich bezüglich ihrer Präfixe unterscheiden.

## Bottom-Up-Lattice-Traversierung
Der rekursive Algorithmus erledigt nun die folgenden Schritte:
- Die Itemsets werden mit Hilfe des minimalen Supports gefiltert. (Anhand der Transaktionsliste ist das sehr einfach möglich.) Verbleibende Itemsets werden in das Ergebnis aufgenommen.
- Aus Paaren zweier Itemsets wird nach dem zuvor beschriebenen Schema ein um ein Element größeres Itemset erstellt.
- Mit der Menge der entstandenen, größeren Itemsets wird der Algorithmus rekursiv fortgesetzt.

In [ ]:
def bottom_up(items, min_supp):
    # Items nach minimalem Support filtern.
    filtered_items = { k: v
                       for k, v in items.items()
                       if len(v) >= min_supp }

    # Abbrechen, falls keine Items mehr verbleiben.
    if not filtered_items:
        return

    # Alle verbleibenden Itemsets zurückgeben, da sie
    # bereits den minimalen Support-Wert erfüllen.
    yield from filtered_items.keys()

    # Über alle verbleibenden Items iterieren.
    for item1, item1_tids in filtered_items.items():
        # Neues Dictionary für neues Präfix erstellen.
        new_items = {}

        # Über alle verbleibenden Items iterieren, um
        # Paare zu bilden.
        for item2, item2_tids in filtered_items.items():
            # Überspringen, falls Paar nicht geordnet ist,
            # um doppelte Behandlung zu vermeiden.
            if item1 >= item2:
                continue

            # Paarung hinsichtlich des Präfix überprüfen.
            # Überspringen, falls Präfix nicht übereinstimmt.
            prefix_len = len(item1) - 1
            if item1[:prefix_len] != item2[:prefix_len]:
                continue

            # Neuen Eintrag anhand der zusammengesetzten
            # Itemsets bilden. Die gemeinsamen Transaktionen
            # werden durch einen Schnitt gefunden.
            new_items[item1 + item2[prefix_len:]] = item1_tids & item2_tids

        # Rekursiver Aufruf mit gefundenen Kandidaten.
        yield from bottom_up(new_items, min_supp)

Die folgende Zelle ruft den Algorithmus mit den erzeugten $1$-Itemsets auf.

In [ ]:
for fi in bottom_up(all_items, min_supp=min_supp):
    print(fi)